In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from plotnine import ggplot, aes, geom_jitter, theme_bw, labs, scale_color_manual, ggsave
import re

In [2]:
file = 'SAV/output/SAV_DataUsed.txt'
SAV4 = pd.read_csv(file, sep='|')

C:\Users\Hill_T\AppData\Local\Temp\ipykernel_8512\1012627805.py:2: DtypeWarning: Columns (15,17,23) have mixed types. Specify dtype option on import or set low_memory=False.


In [3]:
SAV4.ManagedAreaName.unique()

array(['Biscayne Bay', 'Florida Keys NMS', 'Alligator Harbor',
       'Apalachicola NERR', 'Rookery Bay', 'Rookery Bay NERR',
       'St. Andrews', 'Pinellas County', 'Big Bend Seagrasses',
       'Apalachicola Bay', 'Boca Ciega Bay', 'Fort Pickens',
       'Loxahatchee River-Lake Worth Creek',
       'Jensen Beach to Jupiter Inlet',
       'Indian River-Malabar to Vero Beach', 'Banana River',
       'Indian River-Vero Beach to Ft. Pierce', 'Mosquito Lagoon',
       'St. Martins Marsh', 'Nature Coast', 'St. Joseph Bay',
       'Pine Island Sound', 'Gasparilla Sound-Charlotte Harbor',
       'Cape Haze', 'Lemon Bay', 'Estero Bay', 'Matlacha Pass',
       'Terra Ceia', 'Cockroach Bay'], dtype=object)

In [4]:
IRVBPAP = SAV4[SAV4.ManagedAreaName=='Indian River-Vero Beach to Ft. Pierce']

In [5]:
SAV4.analysisunit.unique()

array(['Total seagrass', 'Halodule wrightii', 'Halophila spp.',
       'Total_SAV', 'Ruppia maritima', 'Drift algae',
       'Thalassia testudinum', 'Syringodium filiforme', 'Attached algae'],
      dtype=object)

In [6]:
# def addfits_blacktrendlines(models, plot_i, param):
#     aucol = plot_i.data.columns[0]
#     if len(models) == 1:
#         return plot_i + plt.geo_line(data=SAV4[SAV4.ManagedAreaName])

In [7]:
def addfits_blacktrendlines(models, plot_i, param):
    aucol = plot_i.data.columns[0]
    if len(models) == 1:
        return plot_i + plt.geom_line(data=SAV4[(SAV4['ManagedAreaName'] == i) & 
                                                (~pd.isna(eval(p))) & 
                                                (eval(aucol) == np.unique(eval(models[0])['data'][aucol]))],
                                      mapping=plt.aes(x='relyear', y=eval(models[0]).predict(level=0)),
                                      color="#000099", size=0.75, alpha=0.7, inherit_aes=False)

In [8]:
EDA = 'plots'
Analyses = ['BB_pct','PA']

In [9]:
failedmods = pd.DataFrame({'model':[],'error':[]})

In [10]:
SAV4.analysisunit_halid

0                Total seagrass
1                Total seagrass
2             Halodule wrightii
3             Halodule wrightii
4             Halodule wrightii
                  ...          
952476    Syringodium filiforme
952477    Syringodium filiforme
952478    Syringodium filiforme
952479    Syringodium filiforme
952480    Syringodium filiforme
Name: analysisunit_halid, Length: 952481, dtype: object

In [11]:
# Filter and group by analysisunit_halid
props_halid = SAV4[~SAV4['analysisunit_halid'].str.contains('Total|Drift|spp\\.', regex=True, na=False) & ~SAV4['PA'].isna()] \
    .groupby(['ManagedAreaName', 'analysisunit_halid', 'relyear']) \
    .agg(n_P=('PA', 'sum'),
         ntot_PA=('PA', 'count')) \
    .reset_index()

props_halid['prop_P'] = props_halid['n_P'] / props_halid['ntot_PA']

# Filter and group by analysisunit
props = SAV4[~SAV4['analysisunit'].str.contains('Total|Drift|decipiens|engelmannii|johnsonii|Unidentified', regex=True, na=False) & ~SAV4['PA'].isna()] \
    .groupby(['ManagedAreaName', 'analysisunit', 'relyear']) \
    .agg(n_P=('PA', 'sum'),
         ntot_PA=('PA', 'count')) \
    .reset_index()

props['prop_P'] = props['n_P'] / props['ntot_PA']

In [12]:
for m in props_halid.ManagedAreaName.unique():
    mask = props_halid.ManagedAreaName == m
    props_halid.loc[mask, 'n_allsp_P'] = props_halid.loc[mask, 'n_P'].sum()
    props_halid.loc[mask, 'sp_prop'] = props_halid.loc[mask, 'n_P'] / props_halid.loc[mask, 'n_P'].sum()
    props_halid.loc[mask, 'sp_pct'] = props_halid.loc[mask, 'n_P'] / props_halid.loc[mask, 'n_P'].sum() * 100

In [13]:
for m in props.ManagedAreaName.unique():
    mask = props.ManagedAreaName == m
    props.loc[mask, 'n_allsp_P'] = props.loc[mask, 'n_P'].sum()
    props.loc[mask, 'sp_prop'] = props.loc[mask, 'n_P'] / props.loc[mask, 'n_P'].sum()
    props.loc[mask, 'sp_pct'] = props.loc[mask, 'n_P'] / props.loc[mask, 'n_P'].sum() * 100

In [14]:
props_halid

,ManagedAreaName,analysisunit_halid,relyear,n_P,ntot_PA,prop_P,n_allsp_P,sp_prop,sp_pct
0,Alligator Harbor,Halodule wrightii,15,32.0,32,1.0,1673.0,0.019127,1.912732
1,Alligator Harbor,Halodule wrightii,16,12.0,12,1.0,1673.0,0.007173,0.717274
2,Alligator Harbor,Halodule wrightii,17,18.0,18,1.0,1673.0,0.010759,1.075912
3,Alligator Harbor,Halodule wrightii,18,28.0,28,1.0,1673.0,0.016736,1.673640
4,Alligator Harbor,Halodule wrightii,19,30.0,30,1.0,1673.0,0.017932,1.793186
...,...,...,...,...,...,...,...,...,...
2699,Terra Ceia,Thalassia testudinum,27,86.0,86,1.0,2601.0,0.033064,3.306421
2700,Terra Ceia,Thalassia testudinum,28,16.0,16,1.0,2601.0,0.006151,0.615148
2701,Terra Ceia,Unidentified Halophila,22,1.0,1,1.0,2601.0,0.000384,0.038447
2702,Terra Ceia,Unidentified Halophila,26,1.0,1,1.0,2601.0,0.000384,0.038447


In [15]:
props

,ManagedAreaName,analysisunit,relyear,n_P,ntot_PA,prop_P,n_allsp_P,sp_prop,sp_pct
0,Alligator Harbor,Halodule wrightii,15,32.0,32,1.0,1673.0,0.019127,1.912732
1,Alligator Harbor,Halodule wrightii,16,12.0,12,1.0,1673.0,0.007173,0.717274
2,Alligator Harbor,Halodule wrightii,17,18.0,18,1.0,1673.0,0.010759,1.075912
3,Alligator Harbor,Halodule wrightii,18,28.0,28,1.0,1673.0,0.016736,1.673640
4,Alligator Harbor,Halodule wrightii,19,30.0,30,1.0,1673.0,0.017932,1.793186
...,...,...,...,...,...,...,...,...,...
2290,Terra Ceia,Thalassia testudinum,24,81.0,81,1.0,2601.0,0.031142,3.114187
2291,Terra Ceia,Thalassia testudinum,25,50.0,50,1.0,2601.0,0.019223,1.922338
2292,Terra Ceia,Thalassia testudinum,26,86.0,86,1.0,2601.0,0.033064,3.306421
2293,Terra Ceia,Thalassia testudinum,27,86.0,86,1.0,2601.0,0.033064,3.306421


In [16]:
props_halid = props_halid.rename(columns={'analysisunit_halid': 'analysisunit'})

In [17]:
props2 = pd.concat([props_halid, props])

In [18]:
props2 = props2.sort_values(by=["ManagedAreaName", "relyear", "analysisunit"])

In [19]:
props = props2.copy()

In [20]:
spcollist = ["#005396","#0088B1","#00ADAE","#65CCB3","#AEE4C1","#FDEBA8","#F8CD6D","#F5A800","#F17B00"]

In [21]:
usenames = 'common'

In [22]:
if usenames == 'common':
    # Define a mapping of common names to scientific names
    common_to_scientific = {
        "Thalassia testudinum": "Turtle grass",
        "Syringodium filiforme": "Manatee grass",
        "Halodule wrightii": "Shoal grass",
        "Ruppia maritima": "Widgeon grass",
        "Halophila decipiens": "Paddle grass",
        "Halophila engelmannii": "Star grass",
        "Halophila johnsonii": "Johnson's seagrass",
        "Unidentified Halophila": "Unidentified Halophila",
        "Halophila spp.": "Halophila spp.",
        "Total seagrass": "Total seagrass",
        "Attached algae": "Attached algae",
        "Drift algae": "Drift algae",
        "Total SAV": "Total SAV"
    }

    # Replace values in "analysisunit_halid" column with common names
    SAV4["analysisunit_halid"] = SAV4["analysisunit_halid"].map(common_to_scientific)
    # Replace values in "analysisunit" column with common names
    SAV4["analysisunit"] = SAV4["analysisunit"].map(common_to_scientific)

    # Define the order of the levels for the "analysisunit" column
    analysisunit_levels = [
        "Unidentified Halophila",
        "Halophila spp.",
        "Johnson's seagrass",
        "Manatee grass",
        "Paddle grass",
        "Shoal grass",
        "Star grass",
        "Turtle grass",
        "Widgeon grass",
        "Attached algae"
    ]

    # Convert "analysisunit" column to a categorical variable with specified levels
    SAV4["analysisunit"] = pd.Categorical(SAV4["analysisunit"], categories=analysisunit_levels)

In [23]:
# Create a sequential color palette using seaborn
prcollist_a = sns.color_palette("YlOrRd", n_colors=len(pd.unique(SAV4["ProgramName"])))
# Create another sequential color palette with reversed colors using seaborn
prcollist_b = sns.color_palette("YlGnBu_r", n_colors=len(pd.unique(SAV4["ProgramName"])))

# Combine the two color palettes alternately
prcollist = [color for i in range(len(prcollist_a)) for color in [prcollist_a[i], prcollist_b[i]]]
# Reverse the order of the colors
prcollist = prcollist[::-1]

# Set a seed for reproducibility
np.random.seed(4691)

# Get unique program names and shuffle them
progs = pd.unique(SAV4["ProgramName"])
np.random.shuffle(progs)

# Create a dictionary with program names as keys and corresponding colors from the palette as values
prcols = dict(zip(progs, prcollist))

# Convert the "column", "name", and "type" lists to a pandas DataFrame
parameters = pd.DataFrame({
    "column": ["BB_all", "BB_pct", "PC", "PO", "PA"],
    "name": ["Braun Blanquet score", "Median percent cover", "Visual percent cover", "Percent occurrence", "Frequency of occurrence"],
    "type": ["BBall", "BBpct", "PC", "PO", "PA"]
})

# Convert the "column" column to strings
parameters["column"] = parameters["column"].astype(str)

In [24]:
ma_halspp = ["Banana River", "Indian River-Malabar to Vero Beach", "Indian River-Vero Beach to Ft. Pierce", "Jensen Beach to Jupiter Inlet",
               "Loxahatchee River-Lake Worth Creek", "Mosquito Lagoon", "Biscayne Bay", "Florida Keys NMS"]

In [25]:
import time, random
start_time = time.time()
n=0
seed=352
random.seed(seed)

In [26]:
spp = ["Halodule wrightii", "Halophila decipiens", "Halophila engelmannii", "Halophila johnsonii", 
         "Halophila spp.", "Ruppia maritima", "Syringodium filiforme", "Thalassia testudinum", "Attached algae"]

spp_common = ["Halophila spp.", "Johnson's seagrass", "Manatee grass", "Paddle grass", 
                "Shoal grass", "Star grass", "Turtle grass", "Widgeon grass", "Attached algae"]

if usenames == "common":
    spcols = dict(zip(spp_common, spcollist))
else:
    spcols = dict(zip(spp, spcollist))

## SCRIPT START ##

In [29]:
SAV4[SAV4.ManagedAreaName=='Alligator Harbor']

,RowID,ProgramID,ProgramName,ParameterName,LocationID,ProgramLocationID,SampleDate,Year,Month,ManagedAreaName,...,PC,PO,SC,PA,BB_all,BB_pct,method,relyear,analysisunit_halid,analysisunit
11,99,558,Franklin County Coastal Waters Seagrass Monito...,Percent Cover,295196,TPR18,2017-07-21 00:00:00.000,2017,7,Alligator Harbor,...,15.0,NaN,NaN,1.0,2.0,15.0,Percent Cover,23,Shoal grass,Shoal grass
13,101,558,Franklin County Coastal Waters Seagrass Monito...,Percent Cover,295196,TPR18,2013-07-25 00:00:00.000,2013,7,Alligator Harbor,...,5.0,NaN,NaN,1.0,1.0,2.5,Percent Cover,19,Shoal grass,Shoal grass
155,886,558,Franklin County Coastal Waters Seagrass Monito...,Percent Cover,295188,TPR04,2009-06-09 00:00:00.000,2009,6,Alligator Harbor,...,70.0,NaN,NaN,1.0,4.0,62.5,Percent Cover,15,Manatee grass,Manatee grass
156,887,558,Franklin County Coastal Waters Seagrass Monito...,Percent Cover,295188,TPR04,2012-07-18 00:00:00.000,2012,7,Alligator Harbor,...,24.0,NaN,NaN,1.0,2.0,15.0,Percent Cover,18,Manatee grass,Manatee grass
157,888,558,Franklin County Coastal Waters Seagrass Monito...,Percent Cover,295188,TPR04,2009-06-09 00:00:00.000,2009,6,Alligator Harbor,...,70.0,NaN,NaN,1.0,4.0,62.5,Percent Cover,15,Manatee grass,Manatee grass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680851,1999526,557,Central Panhandle Aquatic Preserves Seagrass M...,Braun Blanquet Score,75428,AH02,2017-07-26 00:00:00.000,2017,7,Alligator Harbor,...,NaN,NaN,NaN,1.0,5.0,87.5,Braun Blanquet,23,Turtle grass,Turtle grass
680984,1999865,557,Central Panhandle Aquatic Preserves Seagrass M...,Braun Blanquet Score,75429,AH03,2018-09-20 00:00:00.000,2018,9,Alligator Harbor,...,NaN,NaN,NaN,1.0,2.0,15.0,Braun Blanquet,24,Turtle grass,Turtle grass
680985,1999866,557,Central Panhandle Aquatic Preserves Seagrass M...,Braun Blanquet Score,75429,AH03,2022-08-03 00:00:00.000,2022,8,Alligator Harbor,...,NaN,NaN,NaN,1.0,5.0,87.5,Braun Blanquet,28,Turtle grass,Turtle grass
681018,1999947,557,Central Panhandle Aquatic Preserves Seagrass M...,Braun Blanquet Score,75430,AH04,2020-10-16 00:00:00.000,2020,10,Alligator Harbor,...,NaN,NaN,NaN,1.0,5.0,87.5,Braun Blanquet,26,Manatee grass,Manatee grass


In [30]:
parameters.column

,column,name,type
0,BB_all,Braun Blanquet score,BBall
1,BB_pct,Median percent cover,BBpct
2,PC,Visual percent cover,PC
3,PO,Percent occurrence,PO
4,PA,Frequency of occurrence,PA


In [27]:
for p in parameters.column:
    print(f'Starting indicator: {p}')
    nyears = SAV4[~SAV4[p].isna() & ~SAV4.analysisunit.isna()].groupby(['ManagedAreaName', 'analysisunit'])['Year'].nunique().reset_index()
    nyears2 = SAV4[~SAV4[p].isna() & ~SAV4.analysisunit_halid.isna()].groupby(['ManagedAreaName','analysisunit_halid'])['Year'].nunique().reset_index()
    
    nyears2.rename(columns={'analysisunit_halid':'analysisunit'}, inplace=True)
    nyears = pd.concat([nyears, nyears2]).drop_duplicates()
    
    ma_include = nyears[nyears.Year>=5].ManagedAreaName.unique()
    for i in ma_include:
        subset_sav4 = SAV4[SAV4.ManagedAreaName==i]
        bbscores_i = subset_sav4.groupby(['analysisunit','BB_all']).size().reset_index(name='n')
        species = bbscores_i[bbscores_i.duplicated('analysisunit', keep=False)].analysisunit.unique()
        print(f'\n Starting MA: {i}')
        print(bbscores_i)
        
        ### plotting ###
        backslash_char= "\\"

        def convert_to_title(i):
            if re.search("NERR", i):
                return re.sub(r'^(.*?)\sNational\sEstuarine\sResearch\sReserve$', r'\1', i) + " National Estuarine Research Reserve"
            elif re.search("NMS", i):
                return re.sub(r'^(.*?)\sNational\sMarine\sSanctuary$', r'\1', i) + " National Marine Sanctuary"
            else:
                return i + " Aquatic Preserve"
        
        def plot_data(EDA, i, p, SAV4, parameters, spcols):
            filtered_data = SAV4[(SAV4['ManagedAreaName'] == i) & (~SAV4[p].isna())]
            unique_species = filtered_data['analysisunit'].unique()
            valid_species = [name for name in spcols.keys() if name in unique_species]
            
            filtered_data = filtered_data[filtered_data['analysisunit'].isin(valid_species)]
            
            param_name = parameters[parameters['column'] == p]['name'].values[0]
            fig, ax = plt.subplots()
            for name, group in filtered_data.groupby('analysisunit'):
                ax.scatter(group['Year'] + np.random.uniform(-0.1, 0.1, size=len(group)), group[p], label=name)
            
            title = convert_to_title(i)
            ax.set_title(title)
            ax.set_xlabel('Year')
            ax.set_ylabel(param_name)
            ax.legend(title='Species')
            plt.style.use('ggplot')
            plt.show()

            return fig
        
#         plot_data(EDA, i, p, SAV4, parameters, spcols)
        


Starting indicator: BB_all

 Starting MA: Alligator Harbor
              analysisunit  BB_all    n
0   Unidentified Halophila     1.0    0
1   Unidentified Halophila     2.0    0
2   Unidentified Halophila     3.0    0
3   Unidentified Halophila     4.0    0
4   Unidentified Halophila     5.0    0
5           Halophila spp.     1.0    1
6           Halophila spp.     2.0    0
7           Halophila spp.     3.0    0
8           Halophila spp.     4.0    0
9           Halophila spp.     5.0    0
10      Johnson's seagrass     1.0    0
11      Johnson's seagrass     2.0    0
12      Johnson's seagrass     3.0    0
13      Johnson's seagrass     4.0    0
14      Johnson's seagrass     5.0    0
15           Manatee grass     1.0  263
16           Manatee grass     2.0  256
17           Manatee grass     3.0  146
18           Manatee grass     4.0   89
19           Manatee grass     5.0   18
20            Paddle grass     1.0    0
21            Paddle grass     2.0    0
22            Paddle 


 Starting MA: Big Bend Seagrasses
              analysisunit  BB_all     n
0   Unidentified Halophila     1.0     0
1   Unidentified Halophila     2.0     0
2   Unidentified Halophila     3.0     0
3   Unidentified Halophila     4.0     0
4   Unidentified Halophila     5.0     0
5           Halophila spp.     1.0   244
6           Halophila spp.     2.0   236
7           Halophila spp.     3.0    53
8           Halophila spp.     4.0    25
9           Halophila spp.     5.0    14
10      Johnson's seagrass     1.0     0
11      Johnson's seagrass     2.0     0
12      Johnson's seagrass     3.0     0
13      Johnson's seagrass     4.0     0
14      Johnson's seagrass     5.0     0
15           Manatee grass     1.0   531
16           Manatee grass     2.0   990
17           Manatee grass     3.0   898
18           Manatee grass     4.0   568
19           Manatee grass     5.0   297
20            Paddle grass     1.0     0
21            Paddle grass     2.0     0
22            Paddle g


 Starting MA: Cockroach Bay
              analysisunit  BB_all    n
0   Unidentified Halophila     0.0    0
1   Unidentified Halophila     1.0    0
2   Unidentified Halophila     2.0    0
3   Unidentified Halophila     3.0    0
4   Unidentified Halophila     4.0    0
5   Unidentified Halophila     5.0    0
6           Halophila spp.     0.0    0
7           Halophila spp.     1.0    5
8           Halophila spp.     2.0    0
9           Halophila spp.     3.0    0
10          Halophila spp.     4.0    0
11          Halophila spp.     5.0    0
12      Johnson's seagrass     0.0    0
13      Johnson's seagrass     1.0    0
14      Johnson's seagrass     2.0    0
15      Johnson's seagrass     3.0    0
16      Johnson's seagrass     4.0    0
17      Johnson's seagrass     5.0    0
18           Manatee grass     0.0    0
19           Manatee grass     1.0   23
20           Manatee grass     2.0   35
21           Manatee grass     3.0   31
22           Manatee grass     4.0   50
23         


 Starting MA: Gasparilla Sound-Charlotte Harbor
              analysisunit  BB_all     n
0   Unidentified Halophila     0.0     0
1   Unidentified Halophila     1.0     0
2   Unidentified Halophila     2.0     0
3   Unidentified Halophila     3.0     0
4   Unidentified Halophila     4.0     0
5   Unidentified Halophila     5.0     0
6           Halophila spp.     0.0     0
7           Halophila spp.     1.0     2
8           Halophila spp.     2.0     0
9           Halophila spp.     3.0     0
10          Halophila spp.     4.0     0
11          Halophila spp.     5.0     0
12      Johnson's seagrass     0.0     0
13      Johnson's seagrass     1.0     0
14      Johnson's seagrass     2.0     0
15      Johnson's seagrass     3.0     0
16      Johnson's seagrass     4.0     0
17      Johnson's seagrass     5.0     0
18           Manatee grass     0.0     0
19           Manatee grass     1.0    57
20           Manatee grass     2.0    37
21           Manatee grass     3.0    36
22      


 Starting MA: Lemon Bay
              analysisunit  BB_all    n
0   Unidentified Halophila     0.0    0
1   Unidentified Halophila     1.0    0
2   Unidentified Halophila     2.0    0
3   Unidentified Halophila     3.0    0
4   Unidentified Halophila     4.0    0
5   Unidentified Halophila     5.0    0
6           Halophila spp.     0.0    0
7           Halophila spp.     1.0    0
8           Halophila spp.     2.0    0
9           Halophila spp.     3.0    0
10          Halophila spp.     4.0    0
11          Halophila spp.     5.0    0
12      Johnson's seagrass     0.0    0
13      Johnson's seagrass     1.0    0
14      Johnson's seagrass     2.0    0
15      Johnson's seagrass     3.0    0
16      Johnson's seagrass     4.0    0
17      Johnson's seagrass     5.0    0
18           Manatee grass     0.0    0
19           Manatee grass     1.0   58
20           Manatee grass     2.0   30
21           Manatee grass     3.0   29
22           Manatee grass     4.0   15
23           Ma

...


 Starting MA: Cockroach Bay
              analysisunit  BB_all    n
0   Unidentified Halophila     0.0    0
1   Unidentified Halophila     1.0    0
2   Unidentified Halophila     2.0    0
3   Unidentified Halophila     3.0    0
4   Unidentified Halophila     4.0    0
5   Unidentified Halophila     5.0    0
6           Halophila spp.     0.0    0
7           Halophila spp.     1.0    5
8           Halophila spp.     2.0    0
9           Halophila spp.     3.0    0
10          Halophila spp.     4.0    0
11          Halophila spp.     5.0    0
12      Johnson's seagrass     0.0    0
13      Johnson's seagrass     1.0    0
14      Johnson's seagrass     2.0    0
15      Johnson's seagrass     3.0    0
16      Johnson's seagrass     4.0    0
17      Johnson's seagrass     5.0    0
18           Manatee grass     0.0    0
19           Manatee grass     1.0   23
20           Manatee grass     2.0   35
21           Manatee grass     3.0   31
22           Manatee grass     4.0   50
23         


 Starting MA: Indian River-Malabar to Vero Beach
              analysisunit  BB_all      n
0   Unidentified Halophila     0.0      0
1   Unidentified Halophila     1.0      0
2   Unidentified Halophila     2.0      0
3   Unidentified Halophila     3.0      0
4   Unidentified Halophila     4.0      0
5   Unidentified Halophila     5.0      0
6           Halophila spp.     0.0  42147
7           Halophila spp.     1.0   2513
8           Halophila spp.     2.0    285
9           Halophila spp.     3.0     94
10          Halophila spp.     4.0     22
11          Halophila spp.     5.0      8
12      Johnson's seagrass     0.0      0
13      Johnson's seagrass     1.0      0
14      Johnson's seagrass     2.0      0
15      Johnson's seagrass     3.0      0
16      Johnson's seagrass     4.0      0
17      Johnson's seagrass     5.0      0
18           Manatee grass     0.0  14008
19           Manatee grass     1.0    585
20           Manatee grass     2.0    168
21           Manatee grass


 Starting MA: Matlacha Pass
              analysisunit  BB_all    n
0   Unidentified Halophila     0.0    0
1   Unidentified Halophila     1.0    0
2   Unidentified Halophila     2.0    0
3   Unidentified Halophila     3.0    0
4   Unidentified Halophila     4.0    0
5   Unidentified Halophila     5.0    0
6           Halophila spp.     0.0    0
7           Halophila spp.     1.0   20
8           Halophila spp.     2.0    4
9           Halophila spp.     3.0    1
10          Halophila spp.     4.0    0
11          Halophila spp.     5.0    0
12      Johnson's seagrass     0.0    0
13      Johnson's seagrass     1.0    0
14      Johnson's seagrass     2.0    0
15      Johnson's seagrass     3.0    0
16      Johnson's seagrass     4.0    0
17      Johnson's seagrass     5.0    0
18           Manatee grass     0.0    0
19           Manatee grass     1.0    0
20           Manatee grass     2.0    0
21           Manatee grass     3.0    0
22           Manatee grass     4.0    0
23         


 Starting MA: Pinellas County
              analysisunit  BB_all     n
0   Unidentified Halophila     0.0     0
1   Unidentified Halophila     1.0     0
2   Unidentified Halophila     2.0     0
3   Unidentified Halophila     3.0     0
4   Unidentified Halophila     4.0     0
5   Unidentified Halophila     5.0     0
6           Halophila spp.     0.0   427
7           Halophila spp.     1.0    80
8           Halophila spp.     2.0     8
9           Halophila spp.     3.0     4
10          Halophila spp.     4.0     0
11          Halophila spp.     5.0     0
12      Johnson's seagrass     0.0     0
13      Johnson's seagrass     1.0     0
14      Johnson's seagrass     2.0     0
15      Johnson's seagrass     3.0     0
16      Johnson's seagrass     4.0     0
17      Johnson's seagrass     5.0     0
18           Manatee grass     0.0   578
19           Manatee grass     1.0   426
20           Manatee grass     2.0   592
21           Manatee grass     3.0   629
22           Manatee grass


 Starting MA: Terra Ceia
              analysisunit  BB_all    n
0   Unidentified Halophila     1.0    0
1   Unidentified Halophila     2.0    0
2   Unidentified Halophila     3.0    0
3   Unidentified Halophila     4.0    0
4   Unidentified Halophila     5.0    0
5           Halophila spp.     1.0    1
6           Halophila spp.     2.0    0
7           Halophila spp.     3.0    1
8           Halophila spp.     4.0    0
9           Halophila spp.     5.0    1
10      Johnson's seagrass     1.0    0
11      Johnson's seagrass     2.0    0
12      Johnson's seagrass     3.0    0
13      Johnson's seagrass     4.0    0
14      Johnson's seagrass     5.0    0
15           Manatee grass     1.0   16
16           Manatee grass     2.0  127
17           Manatee grass     3.0  139
18           Manatee grass     4.0  124
19           Manatee grass     5.0   86
20            Paddle grass     1.0    0
21            Paddle grass     2.0    0
22            Paddle grass     3.0    0
23            

In [141]:
def plot_data(EDA, i, p, SAV4, parameters, spcols):
    filtered_data = SAV4[(SAV4['ManagedAreaName'] == i) & (~SAV4[p].isna())]
    unique_species = filtered_data['analysisunit'].unique()
    valid_species = [name for name in spcols.keys() if name in unique_species]

    filtered_data = filtered_data[filtered_data['analysisunit'].isin(valid_species)]
    by_year = filtered_data.groupby(['Year','analysisunit_halid']).PA.mean()

    param_name = parameters[parameters['column'] == p]['name'].values[0]
    
    fig = plt.bar(by_year.index[i][0], filtered_data[p])
    plt.show()
    return fig

In [172]:
filtered_data = SAV4[(SAV4['ManagedAreaName'] == 'Indian River-Vero Beach to Ft. Pierce') & (~SAV4.PA.isna())]

unique_species = filtered_data['analysisunit'].unique()
valid_species = [name for name in spcols.keys() if name in unique_species]

filtered_data = filtered_data[filtered_data['analysisunit'].isin(valid_species)]
plt.bar(filtered_data.groupby(['Year','analysisunit'])['Year'].mean(), filtered_data.groupby(['Year','analysisunit'])['PA'].mean()*100)

<BarContainer object of 290 artists>

In [201]:
by_year = filtered_data.groupby(['Year','analysisunit']).PA.mean()*100
by_year[2005]

analysisunit
Unidentified Halophila          NaN
Halophila spp.             3.434343
Johnson's seagrass              NaN
Manatee grass             30.909091
Paddle grass                    NaN
Shoal grass               75.151515
Star grass                      NaN
Turtle grass              26.060606
Widgeon grass              9.696970
Attached algae                  NaN
Name: PA, dtype: float64

In [167]:
parameters

,column,name,type
0,BB_all,Braun Blanquet score,BBall
1,BB_pct,Median percent cover,BBpct
2,PC,Visual percent cover,PC
3,PO,Percent occurrence,PO
4,PA,Frequency of occurrence,PA
